In [56]:
!pip install transformers datasets accelerate -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [57]:
import shutil
shutil.rmtree("/kaggle/working/results", ignore_errors=True)

In [ ]:
import pandas as pd
import json

df = pd.read_csv("/kaggle/input/output/output.csv")  

definition_templates = [
    "What does {word} mean?",
    "Can you define {word}?",
    "Tell me the meaning of {word}.",
    "What's the meaning of {word}?",
    "Explain the word {word}."
]
example_templates = [
    "Can you give me an example of {word}?",
    "Show me a sentence using {word}.",
    "How is {word} used in a sentence?",
    "Give an example sentence with {word}.",
    "Use {word} in a sentence."
]

data = []
for _, row in df.iterrows():
    word = str(row["word"])
    definition = str(row["definition"]) if pd.notnull(row["definition"]) else ""
    example = str(row["example"]) if pd.notnull(row["example"]) else ""
    for temp in definition_templates:
        instruction = temp.format(word=word)
        response = f"{word.capitalize()} means {definition}."
        data.append({"instruction": instruction, "response": response})
    for temp in example_templates:
        instruction = temp.format(word=word)
        response = example
        data.append({"instruction": instruction, "response": response})

with open("vocab_data.jsonl", "w", encoding="utf-8") as f:
    for item in data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

In [59]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

torch.cuda.empty_cache()
dataset = load_dataset('json', data_files={'train': 'vocab_data.jsonl'})

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

def preprocess_function(examples):
    input_ids_list = []
    labels_list = []
    max_length = 128
    for prompt, response in zip(examples['instruction'], examples['response']):
        prompt_ids = tokenizer(prompt, truncation=True, max_length=64, add_special_tokens=False)['input_ids']
        response_ids = tokenizer(response, truncation=True, max_length=64, add_special_tokens=False)['input_ids']
        input_ids = prompt_ids + [tokenizer.eos_token_id] + response_ids + [tokenizer.eos_token_id]
        labels = [-100] * (len(prompt_ids) + 1) + response_ids + [tokenizer.eos_token_id]
        input_ids = input_ids[:max_length] + [tokenizer.pad_token_id] * (max_length - len(input_ids))
        labels = labels[:max_length] + [-100] * (max_length - len(labels))
        input_ids_list.append(input_ids)
        labels_list.append(labels)
    return {
        'input_ids': input_ids_list,
        'labels': labels_list,
        'attention_mask': [[1 if id != tokenizer.pad_token_id else 0 for id in ids] for ids in input_ids_list]
    }

train_dataset = dataset['train'].map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [60]:
training_args = TrainingArguments(
    output_dir="./final_model",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    learning_rate=5e-5,
    fp16=True,
    save_strategy="no",
    logging_strategy="no",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

trainer.train()

model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")


/tmp/ipykernel_31/2104615578.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.json',
 './final_model/merges.txt',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("./final_model")
tokenizer = AutoTokenizer.from_pretrained("./final_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def ask_model(prompt):
    formatted = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100, do_sample=False, temperature=0.7)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(formatted, ""))


ask_model("What is the meaning of generation")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generation means A group of people born and living in the same year..
